In [ ]:
from datasets import load_dataset, Dataset
ds  = load_dataset("simplescaling/s1K-1.1")['train']
selected_qhashes = set()
for example in geminiall:
    qhash = question_hash(example['question'])
    if featurized_dict[qhash]['isgeminicorrect']:
        if example['source_type'] in ['Idavidrein/gpqa', 'qq8933/AIME_1983_2024']:
            selected_qhashes.add(qhash)
        elif 'qfq/openaimath' in example['source_type']:
            if featurized_dict[qhash]['gemini_length'] > 5600:
                selected_qhashes.add(qhash)

KeyError: 'validation'

In [10]:
aime = []
for d in ds:
    if 'qfq/openaimath' in d['source_type']:
        aime.append(d)
len(aime)

85

In [9]:
import json
import re
def extract_answer(text):
    # 使用正则表达式匹配\boxed{<answer>}的格式
    match = re.search(r'\\boxed\{(.*?)\}', text)
    if match:
        return match.groups()[-1]  # 提取并返回匹配的内容
    return None

with open("results_dy*/DeepSeek-R1-Distill-Qwen-7B/aime.json", "r") as jf:
    results = json.load(jf)
print(len(results))
# aime = aime[:len(results)]
# for r, d in zip(results, aime):
#     r.update({
#         'question': d['question'],
#         'solution': d['solution']
#     })
nb_correct, b_correct = 0, 0
for r in results:
    # print(r['model_output']['budget_ans'])
    # print("-"*80)
    # print(r['model_output']['budget_ans'])
    # print("="*80)
    no_budget_answer = extract_answer(r['model_output']['no_budget_ans'])
    budget_answer = extract_answer(r['model_output']['budget_ans'])
    # print(no_budget_answer, budget_answer, r['solution'])
    # if no_budget_answer is not None:
    #     print(no_budget_answer)
    if no_budget_answer is not None and no_budget_answer==r['solution']:
        nb_correct+=1
    if budget_answer is not None and budget_answer==r['solution']:
        b_correct+=1
    else:
        print(r['question'])
        print('#'*20)
        print(r['solution'])
        print('*'*20)
        print(r['model_output']['budget_ans'])
        with open("_tmp.json", 'w') as file:
            json.dump({
                'question':r['question'],
                'solution':r['solution'],
                'model_output':r['model_output']['budget_ans'],
            }, file)
        break
# print('no_budget_acc:', nb_correct/len(results))
# print('budget_acc:', b_correct/len(results))

287
Kathy has $5$ red cards and $5$ green cards. She shuffles the $10$ cards and lays out $5$ of the cards in a row in a random order. She will be happy if and only if all the red cards laid out are adjacent and all the green cards laid out are adjacent. For example, card orders RRGGG, GGGGR, or RRRRR will make Kathy happy, but RRRGR will not. The probability that Kathy will be happy is $\frac{m}{n}$ , where $m$ and $n$ are relatively prime positive integers. Find $m + n$ .
####################
157
********************
<|im_start|>user
Kathy has $5$ red cards and $5$ green cards. She shuffles the $10$ cards and lays out $5$ of the cards in a row in a random order. She will be happy if and only if all the red cards laid out are adjacent and all the green cards laid out are adjacent. For example, card orders RRGGG, GGGGR, or RRRRR will make Kathy happy, but RRRGR will not. The probability that Kathy will be happy is $\frac{m}{n}$ , where $m$ and $n$ are relatively prime positive integers

In [ ]:
from collections import Counter
import re
import json

with open("./results/DeepSeek-R1-Distill-Qwen-7B/aime.json", "r") as jf:
    results = json.load(jf)
texts = [(r['model_output']['no_budget_ans']) for r in results]
# Step 1: 合并文本
combined_text = ' '.join(texts).lower()

# Step 2: 去除标点
clean_text = re.sub(r'[^\w\s]', '', combined_text)

# Step 3: 分词
words = clean_text.split()

# Step 4: 统计词频
word_counts = Counter(words)

# Step 5: 找到最高频的词
most_common = word_counts.most_common(5)
    

NameError: name 'json' is not defined

In [3]:
import random
random.choices(range(50), k=10)

[32, 28, 21, 23, 21, 22, 30, 44, 18, 5]

In [13]:
for d in aime:
    print(d['Answer'])

60
15
20
26
4
35
57
61
12
432
288
65
448
130
175
93
592
144
649
512
24
997
160
20
119
106
401
15
38
384
26
198
32
986
315
757
61
49
600
85
182
401
25
864
337
104
150
890
33
981
141
306
358
816
61
560
750
400
300
137
182
480
588
193
70
112
33
120
486
19
373
462
169
27
634
110
364
192
840
987
84
704
869
968
675
283
925
334
144
947
905
528
828
117
432
840
89
73
144
23
184
594
20
146
840
159
128
10
44
532
135
677
990
384
400
502
164
62
660
156
320
819
164
572
945
792
820
94
396
728
580
870
763
495
5
365
118
250
93
344
163
448
997
63
312
315
394
450
465
702
71
597
255
25
67
224
51
589
85
616
215
40
5
400
378
37
340
44
166
23
49
300
799
342
159
276
58
65
768
777
750
125
126
17
417
42
198
90
233
117
241
58
582
554
25
480
800
17
40
308
196
618
87
152
525
83
368
130
761
29
118
38
185
223
314
650
25
259
489
177
345
742
463
408
8
21
667
260
26
997
5
52
25
173
248
177
731
571
927
98
758
180
376
181
110
0
647
131
118
200
495
630
651
500
291
937
79
923
315
61
200
149
5
174
351
85
816
298
898
67
253


In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)
print(input_ids)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )
# response = outputs[0][input_ids.shape[-1]:]
# print(tokenizer.decode(response, skip_special_tokens=True))


tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,  55066,
           6369,   6465,    889,   2744,  31680,    304,  55066,   6604,      0,
         128009, 128006,    882, 128007,    271,  15546,    527,    499,     30,
         128009, 128006,  78191, 128007,    271]])


In [13]:
with open("./output.txt", 'r') as file:
    output = file.read() 
import re
match = re.search(r"\[GUIDE\]\n(Wait, let[’']s[\s\S]+?)(?=\n\[|$)", output)
guide = match.group(1) if match else "Guide not found"
print(guide)

Wait, let's focus on the primary functions of organelles first. For mitochondria, the main function is ATP production through cellular respiration, not protein synthesis.


In [4]:
import re
outputs = ['', '', '', '', '']
solutions = [1,2,3,4,5]
def extract_answer(text):
    # 使用正则表达式匹配\boxed{<answer>}的格式
    match = re.search(r'\\boxed\{(.*?)\}', text)
    if match:
        return match.groups()[-1]  # 提取并返回匹配的内容
eval_results = []
for o, sol in zip(outputs, solutions):
    answer = extract_answer(o)
    if answer is not None and answer==sol:
        eval_results.append(True)
    else:
        eval_results.append(False)
print(eval_results)

[False, False, False, False, False]


In [1]:
import json
from utils import GuidePool, GuideNode
with open("results_sample/DeepSeek-R1-Distill-Qwen-7B/eval/gpqa.json", 'r') as file:
    data = json.load(file)

/home/zli/anaconda3/envs/codellm2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-28 09:52:57 [__init__.py:239] Automatically detected platform cuda.


2025-04-28 09:52:57,553	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
data[100]['eval']

[True, True, False, False, False]

In [ ]:
import json
data = json.load(open("results_sample/DeepSeek-R1-Distill-Qwen-7B/eval/aime.json", "r"))

In [10]:
import json
baselines = ["cons_base", "cons_base"]
data = json.load(open("results_sample/DeepSeek-R1-Distill-Qwen-7B/aime_seleted_by_cons_base.json"))
# eval_data = json.load(open("results_sample/DeepSeek-R1-Distill-Qwen-7B/aime.json"))

In [7]:
# eval_data[0]['solution']
len(eval_data)

90

In [1]:
import json
from utils import load_my_dataset, evaluate, ID_2_MODELS
model_id = 6
model = ID_2_MODELS[model_id].split('/')[-1]
datasets = ["aime", "gpqa", "openaimath"]
baselines = ["logprob_sum", "logprob_avg", "cons_base", "cons_consensus", "cons_weight"]
for dataset in datasets:
    eval_data = json.load(open(f"results_sample**/{model}/eval/{dataset}.json"))
    
    for bl in baselines:
        data = json.load(open(f"results_sample**/{model}/{dataset}_seleted_by_{bl}.json"))
        correct = 0
        for d , ed in zip(data, eval_data):
            # print(ed['eval'])
            chosed_idx = d['scores'].index(max(d['scores']))
            if ed['eval'][chosed_idx]:
                correct+=1
        print("DataName: ", dataset, "\tBaseline: ", bl, "\tAcc: ", correct/len(data))

/home/zli/anaconda3/envs/codellm2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-12 10:41:13 [__init__.py:239] Automatically detected platform cuda.


2025-05-12 10:41:19,887	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


DataName:  aime 	Baseline:  logprob_sum 	Acc:  0.16666666666666666
DataName:  aime 	Baseline:  logprob_avg 	Acc:  0.25555555555555554
DataName:  aime 	Baseline:  cons_base 	Acc:  0.2222222222222222
DataName:  aime 	Baseline:  cons_consensus 	Acc:  0.2
DataName:  aime 	Baseline:  cons_weight 	Acc:  0.2
DataName:  gpqa 	Baseline:  logprob_sum 	Acc:  0.9696969696969697
DataName:  gpqa 	Baseline:  logprob_avg 	Acc:  0.9848484848484849
DataName:  gpqa 	Baseline:  cons_base 	Acc:  0.9696969696969697
DataName:  gpqa 	Baseline:  cons_consensus 	Acc:  0.9747474747474747
DataName:  gpqa 	Baseline:  cons_weight 	Acc:  0.9696969696969697
DataName:  openaimath 	Baseline:  logprob_sum 	Acc:  0.894
DataName:  openaimath 	Baseline:  logprob_avg 	Acc:  0.882
DataName:  openaimath 	Baseline:  cons_base 	Acc:  0.91
DataName:  openaimath 	Baseline:  cons_consensus 	Acc:  0.906
DataName:  openaimath 	Baseline:  cons_weight 	Acc:  0.908


In [ ]:
import json
datasets = ["aime", "gpqa", "openaimath"]
from utils import load_my_dataset, evaluate, ID_2_MODELS
model_id = 6
model = ID_2_MODELS[model_id].split('/')[-1]
for dataset in datasets:
    eval_data = json.load(open(f"results_pt_opt/{model}/opt_0/{dataset}_eval.json"))
    print(sum(eval_data["is_correct"])/len(eval_data["is_correct"]))
 

/home/zli/anaconda3/envs/codellm2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-12 13:32:59 [__init__.py:239] Automatically detected platform cuda.


2025-05-12 13:33:07,286	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


0.5222222222222223
0.898989898989899
0.898


In [1]:
import json
from utils import load_my_dataset, evaluate, ID_2_MODELS
model_id = 2
model = ID_2_MODELS[model_id].split('/')[-1]
with open(f"results_pt_opt/{model}/opt_1/aime_eval.json", 'r') as file:
    results = json.load(file)

print(sum(results['is_correct'])/len(results['is_correct']))

/home/zli/anaconda3/envs/codellm2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-12 11:45:19 [__init__.py:239] Automatically detected platform cuda.


2025-05-12 11:45:32,794	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


0.28888888888888886


In [2]:
import json
from utils import *
datasets = ["aime", "gpqa", "openaimath"]#
model_id = 6
model = ID_2_MODELS[model_id].split('/')[-1]
for dataset in datasets:
    eval_data = json.load(open(f"results_greedy**/{model}/eval/{dataset}.json"))
    eval_list = [ed["eval"] for ed in eval_data]
    # print(len(eval_list))
    print(sum(eval_list)/len(eval_list))
    # if eval_data[0]['solution'] is None:
    # print(eval_data[0].keys())
    # print(eval_data[10]['num_guide'])
    print("-"*100)
    eval_data = json.load(open(f"results_s1**/{model}/eval/{dataset}.json"))
    eval_list = [ed["eval"] for ed in eval_data]
    # print(len(eval_list))
    print(sum(eval_list)/len(eval_list))
    # if eval_data[0]['solution'] is None:
    #     print(dataset, eval_data[0].keys())
    # print(eval_data[0].keys())
    # print(eval_data[10]['num_waits'])
    print("*"*100)

0.17777777777777778
----------------------------------------------------------------------------------------------------
0.17777777777777778
****************************************************************************************************
0.9191919191919192
----------------------------------------------------------------------------------------------------
0.8888888888888888
****************************************************************************************************
0.904
----------------------------------------------------------------------------------------------------
0.9
****************************************************************************************************
